In [3]:
# ==============================
# Standard Library Imports
# ==============================
from enum import Enum
import random
import cma  # Make sure you've added this import for the CMA-ES optimizer

# ==============================
# Third-party Library Imports
# ==============================
import matplotlib.pyplot as plt
from IPython.display import display, Latex
from matplotlib.ticker import MultipleLocator
import numpy as np  # Original numpy
import scipy as sp
import pennylane as qml

# Pennylane numpy
import pennylane.numpy as pnp

# ==============================
# Setup for Quantum Computations
# ==============================
# PennyLane settings
coeffs = [-0.5]
obs = [qml.PauliZ(0)]
hamiltonian = qml.Hamiltonian(coeffs, obs)
dev = qml.device('default.mixed', wires=1)

# Global Variables
Tau_global = 5e-2
Paras_global = pnp.zeros(2)
Phi_global = 0
Gamma_ps = 0

class DataIndex(Enum):
    BEFORE = 0
    PHI = 0
    CFI = 1
    PARAS = 2
    THETA_X = 2
    PHI_Z = 3

def Dephase_factor(tau):
    return 1 - np.exp(-2 * tau)

@qml.qnode(dev)
def circuit(phi):
    global Tau_global, Paras_global
    
    gamma_dephase_in = Dephase_factor(Tau_global)  
    theta_x, phi_z = Paras_global

    qml.RX(pnp.pi/2, wires = 0)
    qml.ApproxTimeEvolution(hamiltonian, phi, 1)
    qml.PhaseDamping(gamma_dephase_in, wires = 0)
    qml.RZ(phi_z, wires = 0)
    qml.RX(theta_x, wires = 0)
    
    return qml.density_matrix(wires = 0)

def Cost_function(paras):
    global Paras_global, Phi_global
    Paras_global = paras
    phi = pnp.array([Phi_global])    
    CFI = qml.qinfo.classical_fisher(circuit)(phi[0])
    return -CFI

In [ ]:
import cma

def CMA_ES(sweep_data, initial_parameters, gamma_ps, iterations):
    """ 
    Perform optimization using CMA-ES.

    Args:
        sweep_data (tuple): (start, end, step) values for the sweep.
        initial_parameters (numpy_array): Array of parameters for the qnode.
        gamma_ps (int): The Gamma value for post-selection.
        iterations (int): Number of iterations for the optimization.

    Returns:
        Numpy array: 3-Dimension Data array containing phi, CFI, and optimized parameters.
    """
    
    # Create Data array
    PHI = np.arange(sweep_data[0], sweep_data[1], sweep_data[2])
    Data = np.zeros((iterations + 1, len(PHI), len(initial_parameters) + 2)) 
    Data[:, :, DataIndex.PHI.value] = PHI.squeeze() # Append PHI in to 0th col
    
    # Set global variables
    global Gamma_ps, Phi_global, Paras_global
    Gamma_ps = gamma_ps 
    
    # Store initial CFI data and parameters
    for idx, phi in enumerate(PHI):
        Data[DataIndex.BEFORE.value][idx][DataIndex.CFI.value] = -Cost_function(initial_parameters)
        Data[DataIndex.BEFORE.value][idx][DataIndex.PARAS.value:] = initial_parameters
        
    # Optimize begin
    for iteration in range(1, iterations + 1):
        for phi_idx, phi_current in enumerate(PHI):
            # Determine initial parameters based on the iteration
            if iteration == 1:
                Paras_Temporary = initial_parameters
            else:
                Paras_Temporary = Data[iteration][phi_idx][DataIndex.PARAS.value:]
            
            # Update the global Phi value
            Phi_global = phi_current
            
            # Determine constraints
            Constraints = get_constraints(phi_current)
            bounds = np.array(Constraints).T
            options = {'bounds': [bounds[0], bounds[1]], 'maxfevals': 1000}
                
            # Optimize the data using CMA-ES
            es = cma.CMAEvolutionStrategy(Paras_Temporary, 0.5, options)
            es.optimize(Cost_function)
            result = es.result

            Data[iteration][phi_idx][DataIndex.CFI.value] = -result.funbest
            Data[iteration][phi_idx][DataIndex.PARAS.value:] = result.xbest
            
    return Data


(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=423678, Thu Oct 26 17:21:10 2023)
NOTE (module=cma, class=CMAEvolutionStrategy, method=tell, iteration=0):  function values=[tensor([[-0.99965821]], requires_grad=True), tensor([[-0.99720951]], requires_grad=True), tensor([[-0.78596979]], requires_grad=True), tensor([[-0.9868676]], requires_grad=True), tensor([[-0.94814649]], requires_grad=True), tensor([[-0.99988033]], requires_grad=True)]


ValueError: objective function values must be a list of scalars

NameError: name 'initial_parameters' is not defined